# LSTM Model(with Dropout, weight tying)
## params:
###### corpus,word_to_id,id_to_word = DataPreprocess(file_addr).creat_corpus_w2d_d2w()

###### N = batch size   
#每个batch_size的长度都是T，但是每个batch的第一个单词的位置会发生变化        
    
###### D = len(word_to_id.items())
    
###### T = len(corpus)-window_size  
#整个时序的长度,与窗口大小有关  
    
###### Ws = hyperParam1           
#window_size，需要向后预测的大小，一般为1 
    
###### Ts = hyperParam2          
#truncated_size，截断反向传播的间距     
    
###### WW_emb = hyperParam3      
#embedding层权重的维度,width of weight of embedding

###### Dr = hyperParam4    
#drop rate的值，取0-1之间的值

###### WW_lstm = Wlf/Wlg/Wli/Wlo = hyperParam5
###### WB_lstm = Blf/Blg/Bli/Blo = hyperParam5    
#LSTM中各种门的权重的维度,Wlf表示width of weight of lstm f gate，Blf表示 width of bias of lstm f gate

## layers:
### Time Embedding ( weight tying with Time Affine)
###### X(N,T)，We(D,WW_emb)→Y1(N,T,WW_emb)   
#X:[[3,2,3,4,7……],……],每个元素是一个单词的id    
### Time Dropout
###### Y1(N,T,WW_emb)，Dr(1)→Y2(N,T,WW_emb)，mask1(N,T,WW_emb)
#dropout丢弃的数据是在哪个维度上？有多种可能，第一种是在batch_size维度，每次丢掉一部分的数据。第二种是在权重的宽度维度上，每次忽略一部分特征的权重，第三种是数据+权重，既丢弃部分数据也丢弃部分权重。还有一些可能是把时间维度也作为丢弃的一部分。    
书中的dropout是在所有的维度上随机丢弃
### Time LSTM1
###### Y2(N,T,WW_emb),    
###### Wl1x(WW_emb,WW_lstm1+WW_lstm1+WW_lstm1+WW_lstm1),
###### Wl1h(WW_lstm1,WW_lstm1+WW_lstm1+WW_lstm1+WW_lstm1),    
###### Bl1(1,WB_lstm1+WB_lstm1+WB_lstm1+WB_lstm1)    
    ↓    
###### Y3(N,T,WW_lstm1)
### Time Dropout
###### Y3(N,T,WW_lstm1)，Dr(1)→Y4(N,T,WW_lstm1)，mask2(N,T,WW_lstm1)
### Time LSTM2
#当存在权重绑定时，需要在输出Affine的前一层，此处为Time LSTM2，对数据维度进行转换，将权重的维度设置成绑定数据的权重维度，才能在下面的Affine实现权重绑定
###### Y4(N,T,WW_lstm1),    
###### Wl2x(WW_lstm1,WW_emb+WW_emb+WW_emb+WW_emb), 
###### Wl2h(WW_emb,WW_emb+WW_emb+WW_emb+WW_emb),    
###### Bl2(1,WW_emb+WW_emb+WW_emb+WW_emb)
    ↓    
###### Y5(N,T,WW_emb)
#### Time Dropout
###### Y5(N,T,WW_emb)，Dr(1)→Y6(N,T,WW_emb)，mask3(N,T,WW_emb)
### Time Affine (weight tying with Time Embedding)
###### Y6(N,T,WW_emb)，We.T(WW_emb,D)，Ba(1,D)→Y7(N,T,D)
### Time Softmax with PerplexityLoss
###### Y7(N,T,D)→Y8(1)    


In [2]:
import numpy as np

In [21]:
class BetterRNN():

    def __init__(self,init_W,init_b,time_T,batch_size,lr=0.01):
        
        We,Wx1,Wh1,Wx2,Wh2 = init_W
        Bl1,Bl2,Ba = init_b
        self.batch_size = batch_size
        self.lr = lr
            
        self.layers = OrderedDict()
        
        self.layers['embedding'] = Embedding(We)
        self.layers['dropout1'] = Dropout()
        self.layers['TimeLSTM1'] = TimeLSTM(Wx1,Wh1,Bl1)
        self.layers['dropout2'] = Dropout()
        self.layers['TimeLSTM2'] = TimeLSTM(Wx2,Wh2,Bl2)
        self.layers['dropout3'] = Dropout()
        self.layers['TimeAffine'] = Affine(We.T,Ba)
        self.layers['TimeSoftmax'] = Softmax()
        self.layers['TimePerPlexityLoss'] = PerPlexityLoss()
        
    def forward(self,X,T_id):
        
        for layer in self.layers.values()[:-1]:
            X = layer.forward(X)
        
        total_loss = self.layers['TimePerPlexityLoss'].forward(X,T_id)
        
        return total_loss
    
    def backward(self,dout=1):
        
        for layer in reversed(self.layers.values()):
            dout = layer.backward(dout)      
        
        return
    
    def renew_params(self):
        
        layer_list = [self.layers['embedding'],
              self.layers['TimeLSTM1'],
              self.layers['TimeLSTM2'],
              self.layers['TimeLSTM3'],
              self.layers['TimeAffine']]
        
        for layer in layer_list:
            for key in layer.params.keys():
                layer.params[key] -= self.lr*layer.grads[key]
                
        return
    
    def train(self,input_corpus,iter_num=100):
        
        loss_list = []
        iter_index_list = []
        
        N = self.batch_size
        T = len(input_corpus)
        
        #生成多个batch_size所需要的数据,batch_data之间最小的间距为1
        batch_dist = max(int(T/N),1)
        X = np.empty((N,T))
        T_id = np.empty((N,T,1))
        
        for i in range(N):
            X[i] = input_corpus[i*batch_dist:]+input_corpus[:i*batch_dist]
            T_id[i] = input_corpus[i*batch_dist+1:]+input_corpus[:i*batch_dist+1].reshape((T,1))
        
        #根据循环次数进行训练
        for n in range(iter_num):
            
            #进行一次前向传播和反向传播
            loss = self.forward(X,T_id)
            self.backward(dout=1)
            
            #根据反向传播导数更新参数
            renew_params()
            
            if n%10 == 0:
                print(f'第{n}次的损失为：',loss)
                loss_list.append(loss)
                iter_index_list.append(n)              
        
        #画图观察训练过程中loss的变化
        fig = plt.figure(figsize=(10,8))
        plt.plot(iter_index_list,loss_list)
        fig.show()
        
        return

In [1]:
class DataPreprocess():
    
    def __init__(self,file_addr,language_type='e'):
        self.file_addr = file_addr
        self.language_type = language_type

    def creat_corpus_w2d_d2w(self):
        '''
        input=string
        output=corpus&word_to_id&id_to_word
        '''
        if self.language_type == 'e':
            
            with open(self.file_addr,'r') as f:
                input_str = f.readlines()
                
            input_str = input_str.lower()
            input_str = input_str.replace('.',' .')

            words = input_str.split(' ')

            word_to_id = {}
            id_to_word = {}

            for word in words:
                if word not in word_to_id:
                    new_id = len(word_to_id)
                    word_to_id[word] = new_id
                    id_to_word[new_id] = word

            corpus = [word_to_id[word] for word in words]
            corpus = np.array(corpus)

            return corpus,word_to_id,id_to_word

In [19]:
class Embedding():
    
    def __init__(self,W):
        
        self.params = {}
        self.params['W'] = W
        
        self.grads = {}
    
    def forward(self,X):
        
        '''
        X(N,len_corpus),W(max_id_in_corpus,WW)
        这里的X是单词id形式，不是one hot形式.
        N是batch_size.
        T是corpus的长度，即需要训练的这段话总共有多少字,也是TimeRNN包含RNN单元的数量值
        WW是分布式向量的维度
        '''
        N,T = X.shape
        MD,WW = self.params['W'].shape
        
        y = np.empty((N,T,WW))
        
        for i in range(N):
            y[i] = W[X[i]]
                
        self.params['X'] = X
                
        return y
    
    def backward(self,dout):
        
        #因为backward中导数存在+=操作，因此每次反向传播开始前，需要先把导数清零
        self.grads['W'] = np.zeros_like(W)
        
        for i in range(N):
            self.grads['W'][X[i]] += dout[i]

        return self.grads['W']

In [3]:
class Lstm():
    
    def __init__(self,Wx,Wh,Bl):
        
        self.params = {}
        self.params['Wx'] = Wx
        self.params['Wh'] = Wh
        self.params['Bl'] = Bl
                
        self.grads = {}
                
    def sigmoid(self,X):
        
        y = 1/(1+np.exp(-1*X))
        
        return y 
    
    def back_sigmoid(self,y,dout):
        
        dx = dout*y(1-y)
        
        return dx 
    
    def tanh(self,X):
        
        y = (np.exp(X) - np.exp(-1*X))/(np.exp(X) + np.exp(-1*X))
        
        return y

    def back_tanh(self,y,dout):
        
        dx = dout*(1-y**2)
        
        return dx
        
    def forward(self,X,H_prev,C_prev):
        
        self.cache = []
                
        #lstm有4个门，Wx是4个门拼合在一起的权重组合，单个的门的权重的维度是整体的1/4 
        four_width_of_weight = self.params['Wx'].shape[1]
        width_of_weight = int(four_width_of_weight/4) 
        d = width_of_weight
        
        #计算各个门的输出
        y_tmp = np.dot(X,Wx) + np.dot(H_prev,Wh) + Bl
        
        yf = y_tmp[:,:d]
        yf = self.sigmoid(yf)
        
        yg = y_tmp[:,d:2*d]
        yg = self.tanh(yg)
        
        yi = y_tmp[:,2*d:3*d]
        yi = self.sigmoid(yi)
        
        yo = y_tmp[:,3*d:]
        yo = self.sigmoid(yo)
        
        #输出记忆内容C和隐藏状态H
        C_next = C_prev*yf + yg*yi
        H_next = self.tanh(C_next)*yo
        
        #将中间结果添加到缓存，方便反向传播调用
        self.cache.append(yf)
        self.cache.append(yg)
        self.cache.append(yi)
        self.cache.append(yo)
        self.cache.append(C_prev)
        self.cache.append(C_next)
        self.cache.append(H_prev)
        self.cache.append(X)
        
        return H_next,C_next
        
    def backward(self,dnext_C,dnext_H,dout_H):
        
        #每次反向传播前初始化权重和偏置的导数,读取正向传播过程中的中间结果
        Wx,Wh,Bl = self.params['Wx'],self.params['Wh'],self.params['Bl']
        self.grads['Wx'] = np.zeros_like(Wx)
        self.grads['Wh'] = np.zeros_like(Wh)
        self.grads['Bl'] = np.zeros_like(Bl)              
        
        yf,yg,yi,yo,C_prev,C_next,H_prev,X = self.cache
        
        #C的导数比较简单，只与yf相关   
        d_prev_C = dnext_C*self.yf
        
        d_H = dnext_H + dout_H
        #求其他导数是时候先求四个中间结果y_f,y_g,y_i,y_o的导数
        #d_f，d_g，d_i导数包含两个部分，一是从C_next传回来,一部分是从H_next传到C_next再传回来
        #d_o导数包含一个部分，从H_next传回来
        d_f = dnext_C*C_prev + d_H*yo*C_prev
        d_g = dnext_C*yi + d_H*yo*yi
        d_i = dnext_C*yg + d_H*yo*yg
        d_o = dout*self.tanh(C_next)
        
        #再求激活函数的导数
        d_f = self.back_sigmoid(yf,d_f)
        d_g = self.back_tanh(yg,d_g)
        d_i = self.back_sigmoid(yi,d_i)
        d_o = self.back_sigmoid(yo,d_o)
        
        #np.hstack()的作用是将array在水平方向按顺序进行合并
        d_y_tmp = np.hstack((d_f,d_g,d_i,d_o))
        
        d_Wx = np.dot(X.T,d_y_tmp)
        d_X = np.dot(d_y_tmp,Wx.T)
        
        d_Wh = np.dot(H_prev.T,d_y_tmp)
        d_prev_H = np.dot(d_y_tmp,Wh.T)
        
        d_Bl = np.sum(d_y_tmp,axis=0)
        
        #将最终导数存入属性或返回
        self.grads['Wx'] = d_Wx
        self.grads['Wh'] = d_Wh
        self.grads['Bl'] = d_Bl 
        
        return d_X,d_prev_C,d_prev_H
        

In [6]:
def TimeLSTM():
    
    def __init__(self,Wx,Wh,Bl,T,Ts,statful=False):
        
        '''
        T = time
        Ts = truncated_size 反向传播截断的距离
        statful=False 不进行truncate,即不进行反向传播截断
        '''
        self.params = {}
        self.params['Wx'] = Wx
        self.params['Wh'] = Wh
        self.params['Bl'] = Bl
        
        self.Ts = Ts
        self.statful = statful
            
        self.grads = {}
        self.grads['Wx'] = np.zeros_like(Wx)
        self.grads['Wh'] = np.zeros_like(Wh)
        self.grads['Bl'] = np.zeros_like(Bl)
        
        self.layers = []
        for i in range(T):
            layer = Lstm(Wx,Wh,Bl)
            self.layers.append(layer)
        
    def forward(self,Xs):
        
        #如果输入数据的时间序列长度与初始化的time_lstm时间序列长度不一致，则报错
        Wx =self.params['Wx']
        N,T,D = Xs.shape
        _,H = Wx.shape
        
        self.h = 0 
        self.c = 0
        
        self.cache = []
        
        Hs = np.empty((N,T,H))
        
        if T != len(self.layers):
            print('长度匹配错误，输入数据的时间序列长度与生成的TimeLstm时间序列长度不一致，请检查')
            return

        for t,layer in enumerate(self.layers):
            Xt= Xs[:,t,:]
            H_prev = self.h
            C_prev = self.c
            self.h,self.c, = layer.forward(Xt,H_prev,C_prev)
            Hs[:,t,:] = self.h
        
        self.cache.append(Xs)
        
        return Hs
        
    def backward(self,d_Hs):
        
        Xs = self.cache[0]
        N,T,D = Xs.shape
        
        self.d_h = 0 
        self.d_c = 0
        
        d_Xs = np.empty_like(Xs)
        
        #每次反向传播前初始化权重和偏置的导数,读取正向传播过程中的中间结果
        self.grads['Wx'] = np.empty_like(Wx)
        self.grads['Wh'] = np.empty_like(Wh)
        self.grads['Bl'] = np.empty_like(Bl)
        
        grads=[0,0,0]
        
        for t in reversed(range(T)):
            
            #当使用truncate时，即将statful设置为True时，在固定位置，截断反向传播的H,C的导数
            if self.statful and (T-t+1)%self.Ts == 0:
                self.d_h = 0
                self.d_c = 0            
            dnext_C = self.d_c
            dnext_H = self.d_h
            
            dout_H = d_Hs[:,t,:]
            
            d_Xt,self.d_c,self.d_h = self.layer[t].backward(dnext_C,dnext_H,dout_H)
            
            d_Xs[:,t,:] = d_Xt
            
            grads[0] += self.layer[t].grads['Wx']
            grads[1] += self.layer[t].grads['Wh']
            grads[2] += self.layer[t].grads['Bl']
            
        self.grads['Wx'] = grads[0]
        self.grads['Wh'] = grads[1]
        self.grads['Bl'] = grads[2]
        
        return d_Xs
    

In [7]:
class Dropout():
    
    def __init__(self,drop_rate=0.5,train_flg = True):
        
        self.dr = drop_rate
        self.cache = []
        self.train_flg = train_flg  #只在训练的时候用dropout防止过拟合，当预测的时候不用dropout
        
    def forward(self,X):
        
        if self.train_flg:
            
            #np.random.rand()根据输入形状返回一个[0,1)之间的【均匀】分布，randn()返回的是[0,1)之间的【正态】分布
            self.mask = np.random.rand(*X.shape)<self.dr

            #这里除以1-self.dr的原因是，为了在drop掉一部分数据之后，整个数据仍然能保持期望值不变。期望值如果发生了改变，整个数据相当于发生了平移，对最终的预测结果有影响
            self.mask = self.mask/(1-self.dr)
        
        else:
            
            self.mask = np.ones_like(X)
            
        return X*self.mask 
    
    def backward(self,dout):
        
        dx = dout*self.mask
        
        return dx

In [16]:
class TimeAffine():
    
    def __init__(self,Wa,Ba):
        
        self.params = {}
        self.params['Wa'] = Wa
        self.params['Ba'] = Ba
        
        self.grads = {}
        self.grads['Wa'] = np.zeros_like(Wa)
        self.grads['Ba'] = np.zeros_like(Ba)
    
    def forward(self,X):
        
        Wa,Ba = self.params['Wa'],self.params['Ba']
        N,T,H = X.shape
        H,D = Wa.shape

        #全链接层的权重只有两个维度，而输入的X包含N,T,H三个维度，需要设置两个中间变量X_tmp和y_tmp来进行转换
        X_tmp = X.reshape((N*T,H))
        self.params['X_tmp'] = X_tmp

        y_tmp = np.dot(X_tmp,Wa) + Ba
        
        y = y_tmp.reshape((N,T,D))
        
        return y
    
    def backward(self,dout):
        
        Wa,Ba,X_tmp = self.params['Wa'],self.params['Ba'],self.params['X_tmp']
        N,T,D = dout.shape
        H,D = Wa.shape
        
        #反向传播与前向传播同理，而输入的dout是三个维度的，无法直接与W.T和X.T进行矩阵点乘，所以同样也需要进行中间转换
        dout_tmp = dout.reshape((N*T,D))
        
        dx_tmp = np.dot(dout_tmp,Wa.T)
        dx = dx_tmp.reshape((N,T,H))
        
        self.grads['Wa'] = np.dot(X_tmp.T,dout_tmp)
        self.grads['Ba'] = np.sum(dout_tmp,axis=0)
        
        return dx

In [17]:
class TimeSoftmax():
    
    def __init__(self):
        
        self.params = {}
        self.grads = {}
        
    def __forward(self,X):
        
        self.params['X'] = X
        N,T,D = X.shape
        
        X_tmp = X.reshape((N*T,D))
        y_tmp = np.exp(X_tmp)/np.sum(np.exp(X_tmp),axis=1,keepdims=True).repeat(D,axis=1)
        y = y_tmp.reshape((N,T,D))
        
        #存储y,在反向传播中会用到
        self.params['y'] = y
        
        return y
    
    def backward(self,dout):
        
        y = self.params['y']
        
        #softmax的导数为y(1-y)，其中y为每个元素正向传播的结果
        dx = dout*(y*(1-y))
        
        self.grads['X'] = dx 
        
        return dx

In [18]:
class TimePerPlexityLoss():
    
    def __init__(self,eps = 1e-8):
        
        #因为采用困惑度作为loss衡量标准，为了防止出现0为分母的情况，所以需要加上一个微小数eps
        self.params ={}
        self.grads = {}
        self.eps = eps
        
    def forward(self,X,T_id):
        
        #input数据T_id需要用id的方式表示,T_id.shape = (N,T,1)
        #只需要把每个长度为D的向量的序号为id的那个值拿出来，计算它的倒数，即为这个单词的损失
        #最终损失需要把N*T个单词的损失全加起来之后除以N*T,表示每个单词的平均困惑度。
        #如果完全拟合的情况下，即最终损失最小的值，为1
        
        loss_t = 0
        
        N,T,D = X.shape
        self.params['X'] = X
        self.params['T_id'] = T_id
        
        for i in range(N):
            for j in range(T):
                loss = 1/(X[i,j,T_id[i,j,0]] + self.eps)
                loss_t += loss
        
        loss_t = loss_t/(N*T)
        
        return loss_t
    
    def backward(self,dout=1):
        
        X,T_id = self.params['X'],self.params['T_id']
        N,T,D = X.shape
        
        dx = np.zeros_like(X)
        for i in range(N):
            for j in range(T):
                dx[i,j,T_id[i,j,0]] += dout*(-1/(X[i,j,T_id[i,j,0]]**2))
        
        self.grads['X'] =dx
        
        return dx